In [47]:
# importing libraries
import os
import urllib.parse
import pandas as pd
import requests as req
from selenium import webdriver
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.firefox import GeckoDriverManager

In [21]:
# setting working directory and reading database
wd = os.getcwd()

discography_df = pd.read_csv(wd + "/discografia_enredos.csv")

In [22]:
# carregando variaveis de ambiente
load_dotenv()
CHAVE_API_VAGALUME = os.getenv("CHAVE_API_VAGALUME")

In [23]:
# checando base
discography_df = discography_df.drop(columns = "Unnamed: 0")

In [24]:
# creating function to call vagalume api
def get_lyrics(artist, song):
    
    params = {
        "art": artist,
        "mus": song,
        "apikey": CHAVE_API_VAGALUME
    }
    
    params_encoded = urllib.parse.urlencode(params)
    
    url = "https://api.vagalume.com.br/search.php?" + params_encoded
    
    # replace quote plus with quote %
    # TODO: how to fix this without using replace?
    
    url = url.replace("+", "%20")
    
    response = req.get(url)
    try:
        data = response.json()
        return data["mus"][0]["text"] if "mus" in data.keys() else data
    except ValueError:
        return "API_ERROR" + "_" + str(response.status_code)



In [11]:
# repescagem 1: rodando pra quem caiu por ratelimit (umas 6/7 vezes resolve)
discography_df.loc[:, "letra"] = discography_df.apply(
    lambda x: get_lyrics(artist = x["escola"], song = "Samba Enredo" + " " + str(x["ano"])) if x["letra"] == "API_ERROR_429" else x["letra"], axis = 1
)

discography_df[discography_df["letra"] == "API_ERROR_429"]["letra"].value_counts()

Series([], Name: letra, dtype: int64)

In [14]:
# verificando quantos enredos estao sem letra (tipo n eh str)
discography_df["letra"].apply(type).value_counts()

<class 'str'>    750
Name: letra, dtype: int64

In [13]:
# repescagem 2: rodando com "samba-enredo" ao inves de "Samba Enredo"
discography_df.loc[:, "letra"] = discography_df.apply(
    lambda x: get_lyrics(artist = x["escola"], song = "Samba-Enredo" + " " + str(x["ano"])) if not isinstance(x["letra"], str) else x["letra"], axis = 1
)

In [17]:
# repescagem 2.1: rodando pra quem caiu no ratelimit (1x resolve)
discography_df.loc[:, "letra"] = discography_df.apply(
    lambda x: get_lyrics(artist = x["escola"], song = "Samba-Enredo" + " " + str(x["ano"])) if x["letra"] == "API_ERROR_429" else x["letra"], axis = 1
)

discography_df[discography_df["letra"] == "API_ERROR_429"]["letra"].value_counts()

Series([], Name: letra, dtype: int64)

In [18]:
# verificando quantos enredos estao sem letra (tipo nao eh str)
discography_df["letra"].apply(type).value_counts()

<class 'str'>     660
<class 'dict'>     90
Name: letra, dtype: int64

In [19]:
# checando enredos que ainda nao foram encontrados
falhas = discography_df[~discography_df["letra"].apply(lambda x: isinstance(x, str))]
falhas.loc[:, "letra"] = falhas.loc[:, "letra"].astype(str)

# algumas falhas se dao pq nao encontra o nome das escolas: ex, Caprichosos de Pilares e Unidos de Sao Carlos (a estacio)
falhas["escola"].value_counts()

/tmp/ipykernel_877/354569550.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  falhas.loc[:, "letra"] = falhas.loc[:, "letra"].astype(str)


Caprichosos                  22
Tradição                     15
Unidos de São Carlos         11
Estácio                       9
Santa Cruz                    6
Jacarezinho                   4
Unidos do Cabuçu              3
Tupy de Brás de Pina          2
Rocinha                       2
Portela                       2
Ritmos da escolas             2
Em Cima da Hora               1
Ritmos das escolas            1
Arrastão de Cascadura         1
Unidos de Lucas               1
Unidos de São Lucas           1
Leão de Nova Iguaçu           1
Independentes de Cordovil     1
Mangueira                     1
Renascer                      1
São Clemente                  1
Inocentes                     1
Grande Rio                    1
Name: escola, dtype: int64

In [24]:
# repescagem 3: Samba-enredo ao inves de Samba Enredo ou Samba-Enredo
discography_df.loc[:, "letra"] = discography_df.apply(
    lambda x: get_lyrics(artist = x["escola"], song = "Samba-enredo" + " " + str(x["ano"])) if not isinstance(x["letra"], str) else x["letra"], axis = 1
)


API_ERROR_429    85
Name: letra, dtype: int64

In [26]:
# repescagem 3.1: erro de api
discography_df.loc[:, "letra"] = discography_df.apply(
    lambda x: get_lyrics(artist = x["escola"], song = "Samba-enredo" + " " + str(x["ano"])) if x["letra"] == "API_ERROR_429" else x["letra"], axis = 1
)

discography_df[discography_df["letra"] == "API_ERROR_429"]["letra"].value_counts()

Series([], Name: letra, dtype: int64)

In [16]:
# verificando quantos enredos estao sem letra (tipo nao eh str)
discography_df["letra"].apply(type).value_counts()

<class 'str'>    750
Name: letra, dtype: int64

In [28]:
# repescagem 4: caprichosos de pilares
discography_df.loc[:, "letra"] = discography_df.apply(
    lambda x: get_lyrics(artist = "Caprichosos de pilares", song = "Samba Enredo" + " " + str(x["ano"])) if x["escola"] == "Caprichosos" else x["letra"], axis = 1
)

discography_df[discography_df["letra"] == "API_ERROR_429"]["letra"].value_counts()

Series([], Name: letra, dtype: int64)

In [29]:
# respescagem 4.1: Samba-Enredo ao inves de Samba Enredo
discography_df.loc[:, "letra"] = discography_df.apply(
    lambda x: get_lyrics(artist = "Caprichosos de pilares", song = "Samba-Enredo" + " " + str(x["ano"])) if (x["escola"] == "Caprichosos" and not isinstance(x["letra"], str)) else x["letra"], axis = 1
)

discography_df[discography_df["letra"] == "API_ERROR_429"]["letra"].value_counts()

Series([], Name: letra, dtype: int64)

In [30]:
# checando enredos que ainda nao foram encontrados
falhas = discography_df[~discography_df["letra"].apply(lambda x: isinstance(x, str))]
falhas.loc[:, "letra"] = falhas.loc[:, "letra"].astype(str)

# algumas falhas se dao pq nao encontra o nome das escolas: ex, Caprichosos de Pilares e Unidos de Sao Carlos (a estacio)
falhas["escola"].value_counts()

/tmp/ipykernel_877/354569550.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  falhas.loc[:, "letra"] = falhas.loc[:, "letra"].astype(str)


Tradição                     15
Unidos de São Carlos         11
Estácio                       9
Santa Cruz                    6
Jacarezinho                   4
Unidos do Cabuçu              3
Tupy de Brás de Pina          2
Rocinha                       2
Caprichosos                   2
Portela                       2
Ritmos da escolas             2
Em Cima da Hora               1
Ritmos das escolas            1
Arrastão de Cascadura         1
Unidos de Lucas               1
Unidos de São Lucas           1
Leão de Nova Iguaçu           1
Independentes de Cordovil     1
Mangueira                     1
Renascer                      1
São Clemente                  1
Inocentes                     1
Grande Rio                    1
Name: escola, dtype: int64

In [31]:
# repescagem 5 - GRES tradicao
discography_df.loc[:, "letra"] = discography_df.apply(
    lambda x: get_lyrics(artist = "GRES Tradicao", song = "Samba-Enredo" + " " + str(x["ano"])) if (x["escola"] == "Tradição" and not isinstance(x["letra"], str)) else x["letra"], axis = 1
)

discography_df[discography_df["letra"] == "API_ERROR_429"]["letra"].value_counts()

Series([], Name: letra, dtype: int64)

In [32]:
# checando enredos que ainda nao foram encontrados
falhas = discography_df[~discography_df["letra"].apply(lambda x: isinstance(x, str))]
falhas.loc[:, "letra"] = falhas.loc[:, "letra"].astype(str)

# algumas falhas se dao pq nao encontra o nome das escolas: ex, Caprichosos de Pilares e Unidos de Sao Carlos (a estacio)
falhas["escola"].value_counts()

/tmp/ipykernel_877/354569550.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  falhas.loc[:, "letra"] = falhas.loc[:, "letra"].astype(str)


Unidos de São Carlos         11
Estácio                       9
Santa Cruz                    6
Jacarezinho                   4
Unidos do Cabuçu              3
Tupy de Brás de Pina          2
Rocinha                       2
Caprichosos                   2
Portela                       2
Ritmos da escolas             2
Unidos de São Lucas           1
Em Cima da Hora               1
Ritmos das escolas            1
Arrastão de Cascadura         1
Unidos de Lucas               1
Leão de Nova Iguaçu           1
Independentes de Cordovil     1
Mangueira                     1
Renascer                      1
São Clemente                  1
Inocentes                     1
Grande Rio                    1
Name: escola, dtype: int64

In [33]:
# repescagem 6: estácio de sá/unidos de são carlos
discography_df.loc[:, "letra"] = discography_df.apply(
    lambda x: get_lyrics(artist = "Estácio de Sá", song = "Samba-Enredo" + " " + str(x["ano"])) if (x["escola"] in ["Estácio", "Unidos de São Carlos"] and not isinstance(x["letra"], str)) else x["letra"], axis = 1
)

In [34]:
# checando enredos que ainda nao foram encontrados
falhas = discography_df[~discography_df["letra"].apply(lambda x: isinstance(x, str))]
falhas.loc[:, "letra"] = falhas.loc[:, "letra"].astype(str)

# algumas falhas se dao pq nao encontra o nome das escolas: ex, Caprichosos de Pilares e Unidos de Sao Carlos (a estacio)
falhas["escola"].value_counts()

/tmp/ipykernel_877/354569550.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  falhas.loc[:, "letra"] = falhas.loc[:, "letra"].astype(str)


Santa Cruz                   6
Jacarezinho                  4
Unidos do Cabuçu             3
Rocinha                      2
Caprichosos                  2
Tupy de Brás de Pina         2
Portela                      2
Ritmos da escolas            2
Estácio                      2
Unidos de São Lucas          1
Em Cima da Hora              1
Ritmos das escolas           1
Arrastão de Cascadura        1
Unidos de Lucas              1
Independentes de Cordovil    1
Leão de Nova Iguaçu          1
Mangueira                    1
Renascer                     1
São Clemente                 1
Inocentes                    1
Grande Rio                   1
Name: escola, dtype: int64

In [35]:
# repescagem 7: academicos de santa cruz
discography_df.loc[:, "letra"] = discography_df.apply(
    lambda x: get_lyrics(artist = "Acadêmicos de Santa Cruz", song = x["enredo"].replace(",", "-")) if (x["escola"] == "Santa Cruz" and not isinstance(x["letra"], str)) else x["letra"], axis = 1
)

In [36]:
# checando enredos que ainda nao foram encontrados
falhas = discography_df[~discography_df["letra"].apply(lambda x: isinstance(x, str))]
falhas.loc[:, "letra"] = falhas.loc[:, "letra"].astype(str)

# algumas falhas se dao pq nao encontra o nome das escolas: ex, Caprichosos de Pilares e Unidos de Sao Carlos (a estacio)
falhas["escola"].value_counts()

/tmp/ipykernel_877/354569550.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  falhas.loc[:, "letra"] = falhas.loc[:, "letra"].astype(str)


Jacarezinho                  4
Unidos do Cabuçu             3
Santa Cruz                   2
Rocinha                      2
Caprichosos                  2
Tupy de Brás de Pina         2
Portela                      2
Ritmos da escolas            2
Estácio                      2
Unidos de São Lucas          1
Em Cima da Hora              1
Ritmos das escolas           1
Arrastão de Cascadura        1
Unidos de Lucas              1
Independentes de Cordovil    1
Leão de Nova Iguaçu          1
Mangueira                    1
Renascer                     1
São Clemente                 1
Inocentes                    1
Grande Rio                   1
Name: escola, dtype: int64

In [37]:
# salvando progresso ate agr
discography_df.to_csv("discografia_enredos.csv")

In [26]:
# filtrando so o que nao conseguiu a letra ate agora
discography_df_no_lyrics = discography_df[discography_df["letra"].str.contains("{")]

In [27]:
discography_df_no_lyrics

,escola,enredo,autor,interpete,ano,letra
4,Portela,"""Tronco do Ipê""",Cabana,NaN,1968,"{'type': 'song_notfound', 'art': {'id': '3ade6..."
7,Ritmos das escolas,Ritmos das escolas,Ritmos das escolas,Ritmos das escolas,1968,{'type': 'notfound'}
9,Estácio,"""Visita ao Museu Imperial""",Wanderlei e Jorge Canário,Jorge Cabo,1968,"{'type': 'song_notfound', 'art': {'id': '3ade6..."
11,Independentes de Cordovil,"""Aspecto da Vida Carioca no Século XVIII""",Alexandre e Ivan,Lucio Martins,1968,"{'type': 'song_notfound', 'art': {'id': '3ade6..."
17,Portela,"""Tronco do Ipê""",Cabana,NaN,1968,"{'type': 'song_notfound', 'art': {'id': '3ade6..."
22,Ritmos da escolas,Ritmos da escolas,Ritmos da escolas,Ritmos da escolas,1969,{'type': 'notfound'}
32,Jacarezinho,"""O Fabuloso Mundo do Circo""",Marcos e Sarabanda,NaN,1970,{'type': 'notfound'}
36,Santa Cruz,"""Bravura, Amor e Beleza da Mulher Brasileira""",Fausto e Paulo,NaN,1970,"{'type': 'song_notfound', 'art': {'id': '3ade6..."
40,Santa Cruz,"""Bravura, Amor e Beleza da Mulher Brasileira""",Fausto e Paulo,NaN,1970,"{'type': 'song_notfound', 'art': {'id': '3ade6..."
41,Unidos de São Lucas,"""Terra de Caruaru""",Corvina e Sidney da Conceição,NaN,1970,"{'type': 'song_notfound', 'art': {'id': '3ade6..."


In [50]:
#oq sobra, apelar pra selenium
driver = webdriver.Firefox(GeckoDriverManager().install())


#for x in range(len(discography_df_no_lyrics)):
#    driver.get("https://www.letras.mus.br/")
#
#    search_bar = driver.find_element(By.ID, "main_suggest")
#    search_bar.send_keys(str(discography_df_no_lyrics.loc[x, "escola"]) + discography_df_no_lyrics[x, "enredo"])
#    break

/tmp/ipykernel_1070/1862990558.py:2: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  driver = webdriver.Firefox(GeckoDriverManager().install())


NotADirectoryError: [Errno 20] Not a directory: '/home/ketzer/.wdm/drivers/geckodriver/linux64/v0.33.0/geckodriver'